In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [37]:
uchicago = pd.read_csv("season_plays23.csv").iloc[:, 1:]
w_uchicago = pd.read_csv("w_season_plays23.csv").iloc[:, 1:]

In [38]:
def points_scored(action):
    if "3-pt" in action:
        return 3
    elif "free throw" in action:
        return 1
    else:
        return 2
    
def calculate_scores(df, lineup):
    current = df[df.Lineup == lineup]
    
    makes = current[current.Action.str.contains('made')]
    makes["Points"] = makes.Action.apply(points_scored)

    chicago_scores = makes[makes.Chicago == makes.Team].Points.sum()
    other_scores = makes[makes.Chicago != makes.Team].Points.sum()

    return (chicago_scores, other_scores)

def calculate_play(df, lineup, play):
    current = df[df.Lineup == lineup]
    plays = current[(current.Action.str.contains(play))]

    chicago = plays[(plays.Chicago == plays.Team)].shape[0]
    other = plays[(plays.Chicago != plays.Team)].shape[0]
    
    return (chicago, other)

def calculate_makes(df, lineup, three=False):
    current = df[df.Lineup == lineup]
    
    makes = current[(current.Action.str.contains("made")) & ~(current.Action.str.contains("free throw"))]
    if three:
        makes = makes[makes.Action.str.contains("3-pt")]
    
    chicago_makes = makes[(makes.Chicago == makes.Team)].shape[0]
    other_makes = makes[(makes.Chicago != makes.Team)].shape[0]
    
    return (chicago_makes, other_makes)

def calculate_percentage(df, lineup, three=False, ft=False, att=False):
    current = df[df.Lineup == lineup]
    misses = current[(current.Action.str.contains("missed"))]
    makes = current[(current.Action.str.contains("made"))]
    
    if three:
        misses = misses[misses.Action.str.contains("3-pt")]
        makes = makes[makes.Action.str.contains("3-pt")]
    elif ft:
        misses = misses[misses.Action.str.contains("free throw")]
        makes = makes[makes.Action.str.contains("free throw")]
    else:
        misses = misses[~(misses.Action.str.contains("free throw"))]
        makes = makes[~(makes.Action.str.contains("free throw"))]
        
    chicago_misses = misses[(misses.Chicago == misses.Team)].shape[0]
    chicago_makes = makes[(makes.Chicago == makes.Team)].shape[0]
    other_misses = misses[(misses.Chicago != misses.Team)].shape[0]
    other_makes = makes[(makes.Chicago != makes.Team)].shape[0]
    
    chicago_attempts = chicago_misses + chicago_makes
    other_attempts = other_misses + other_makes
    
    if att:
        return (chicago_attempts, other_attempts)
    
    if (chicago_attempts == 0) and (other_attempts == 0):
        return (0, 0)
    elif chicago_attempts == 0:
        return (0, other_makes / other_attempts)
    elif other_attempts == 0:
        return (chicago_makes / chicago_attempts, 0)
    else:
        return (chicago_makes / chicago_attempts, 
                other_makes / other_attempts)
    
def sec_to_time(seconds):
    minutes = str(seconds // 60)
    
    if (seconds % 60) < 10:
        return minutes + ":0" + str(seconds % 60)
    else:
        return minutes + ":" + str(seconds % 60)

def time_passed(start, end):
    if start == '30300':
        start = "00:30"
    elif end == '30300':
        end = "00:30"
    
    start_time = int(start.split(":")[0])*60 + int(start.split(":")[1])
    end_time = int(end.split(":")[0])*60 + int(end.split(":")[1])
    
    if start_time > end_time:
        return start_time - end_time
    else:
        return 0

def calculate_time_played(df, lineup):
    total_time = 0
    for x in range(len(df[df.Lineup == lineup])-1):
        current_play = df[df.Lineup == lineup].reset_index(drop=False).values[x]
        next_play = df[df.Lineup == lineup].reset_index(drop=False).values[x+1]
        if x > 0:
            last_play = df[df.Lineup == lineup].reset_index(drop=False).values[x-1]
        else:
            last_play = current_play

        if (current_play[0] != next_play[0] - 1) or (current_play[0] == 342):
            continue

        if (current_play[0] == 0) or (last_play[4] != current_play[4]):
            if (current_play[4] == 3):
                start = "05:00"
            else:
                start = team_start
            end = next_play[5]
        elif next_play[4] != current_play[4]:
            start = current_play[5]
            end = "00:00"
        else:
            start = current_play[5]
            end = next_play[5]

        total_time += time_passed(start, end)

    return total_time / 60

def analyze_lineup(df, lineup):
    return [calculate_time_played(df, lineup), # MP
            
            calculate_scores(df, lineup)[0], # PTS
            calculate_scores(df, lineup)[1], # OPP PTS
            
            calculate_scores(df, lineup)[0] - calculate_scores(df, lineup)[1], # +/-
            
            calculate_play(df, lineup, "Assist")[0], # AST
            
            calculate_play(df, lineup, "offensive rebound")[0], # OREB
            calculate_play(df, lineup, "offensive rebound")[1], # OPP OREB
            calculate_play(df, lineup, "defensive rebound")[0], # DREB
            calculate_play(df, lineup, "defensive rebound")[1], # OPP DREB
            
            calculate_makes(df, lineup)[0], # FGM
            calculate_percentage(df, lineup, att=True)[0], # FGA
            calculate_makes(df, lineup)[1], # OPP FGM
            calculate_percentage(df, lineup, att=True)[1], # OPP FGA
            
            calculate_makes(df, lineup, three=True)[0], # 3PM
            calculate_percentage(df, lineup, three=True, att=True)[0], # 3PA
            calculate_makes(df, lineup, three=True)[1], # OPP 3PM
            calculate_percentage(df, lineup, three=True, att=True)[1], # OPP 3PA
            
            calculate_percentage(df, lineup, ft=True)[0], # FT%
            calculate_percentage(df, lineup, ft=True)[1], # OPP FT%
            calculate_percentage(df, lineup, ft=True, att=True)[0], # FTA
            calculate_percentage(df, lineup, ft=True, att=True)[1], # OPP FTA
            
            calculate_play(df, lineup, "Turnover")[0], # TO
            calculate_play(df, lineup, "Turnover")[1],# OPP TO
            
            calculate_play(df, lineup, "Steal")[0], # STL
            calculate_play(df, lineup, "Block")[0], # BLK
            
            calculate_play(df, lineup, "Foul")[0], # PF
            calculate_play(df, lineup, "Foul")[1]] # OPP PF

def pbp_lineups(game_pbp):
    date = game_pbp.Date.values[0]
    lineups = [l for l in pd.unique(game_pbp.Lineup) if len(l.split(", ")) == 5]

    lineup_data = []
    for l in lineups:
        lineup_data.append( [l] + analyze_lineup(game_pbp, l) )

    lineup_analysis = pd.DataFrame(lineup_data, 
                                   columns = ['Lineup',
                                              'MP', 
                                              'PTS', 'OPP PTS', 
                                              '+/-', 
                                              'AST',
                                              'OREB', 'OPP OREB', 'DREB', 'OPP DREB',
                                              'FGM', 'FGA', 'OPP FGM', 'OPP FGA',
                                              '3PM', '3PA', 'OPP 3PM', 'OPP 3PA',
                                              'FT%', 'OPP FT%', 'FTA', 'OPP FTA', 
                                              'TO', 'OPP TO', 
                                              'STL', 'BLK',
                                              'PF', 'OPP PF'])
    
    lineup_analysis['Lineup_Set'] = lineup_analysis.Lineup.apply(lambda i : set(i.split(", ")))
    lineup_analysis['Lineup'] = lineup_analysis['Lineup_Set'].apply(lambda i : ", ".join(sorted(i)))
    lineup_analysis = lineup_analysis.groupby('Lineup').sum().reset_index(drop=False)
    
    return lineup_analysis

In [39]:
def add_advanced(lineup_analysis):
    lineup_analysis['FG%'] = 100 * lineup_analysis['FGM'] / lineup_analysis['FGA']
    lineup_analysis['OPP FG%'] = 100 * lineup_analysis['FGM'] / lineup_analysis['FGA']
    lineup_analysis['3P%'] = 100 * lineup_analysis['3PM'] / lineup_analysis['3PA']
    lineup_analysis['OPP 3P%'] = 100 * lineup_analysis['OPP 3PM'] / lineup_analysis['OPP 3PA']
    
    # formula from nbastuffer.com
    lineup_analysis['POSS'] = 0.96 * (lineup_analysis.FGA +
                                      lineup_analysis.TO + 
                                      0.44*(lineup_analysis.FTA) - 
                                      lineup_analysis.OREB)
    lineup_analysis['OPP POSS'] = 0.96 * (lineup_analysis['OPP FGA'] +
                                          lineup_analysis['OPP TO'] + 
                                          0.44*(lineup_analysis['OPP FTA']) - 
                                          lineup_analysis['OPP OREB'])

    lineup_analysis['PPP'] = lineup_analysis.PTS / lineup_analysis.POSS
    lineup_analysis['OPP PPP'] = lineup_analysis['OPP PTS'] / lineup_analysis['OPP POSS']
    
    lineup_analysis['FTA%'] = 100 * lineup_analysis['FTA'].astype(int) / lineup_analysis['FGA'].astype(int)
    lineup_analysis['OPP FTA%'] = 100 * lineup_analysis['OPP FTA'].astype(int) / lineup_analysis['OPP FGA'].astype(int)

    lineup_analysis['PTS DIFF PER40'] = 40 * (lineup_analysis['PTS'] - lineup_analysis['OPP PTS']) / lineup_analysis['MP']

    lineup_analysis['REB'] = lineup_analysis.OREB + lineup_analysis.DREB
    lineup_analysis['OPP REB'] = lineup_analysis['OPP OREB'] + lineup_analysis['OPP DREB']
    lineup_analysis['ORB%'] = 100 * lineup_analysis.OREB / (lineup_analysis.OREB + lineup_analysis['OPP DREB'])
    lineup_analysis['DRB%'] = 100 * lineup_analysis.DREB / (lineup_analysis.DREB + lineup_analysis['OPP OREB'])
    lineup_analysis['RB%'] = 100 * lineup_analysis.REB / (lineup_analysis.REB + lineup_analysis['OPP REB'])
    lineup_analysis['OPP ORB%'] = 100 * lineup_analysis['OPP OREB'] / (lineup_analysis['OPP OREB'] + lineup_analysis.DREB)
    lineup_analysis['OPP DRB%'] = 100 * lineup_analysis['OPP DREB'] / (lineup_analysis['OPP DREB'] + lineup_analysis.OREB)
    lineup_analysis['OPP RB%'] = 100 * lineup_analysis['OPP REB'] / (lineup_analysis['OPP REB'] + lineup_analysis.REB)

    lineup_analysis['TO%'] = 100 * lineup_analysis.TO / (lineup_analysis.FGA + 
                                                         0.44*(lineup_analysis.FTA) + 
                                                         lineup_analysis.TO)
    lineup_analysis['OPP TO%'] = 100 * lineup_analysis['OPP TO'] / (lineup_analysis['OPP FGA'] + 
                                                         0.44*(lineup_analysis['OPP FTA']) + 
                                                         lineup_analysis['OPP TO'])
    
    lineup_analysis['POSS PER40'] = 40 * lineup_analysis['POSS'] / lineup_analysis['MP']
    
    lineup_analysis = lineup_analysis.round(2)
    lineup_analysis['POSS'] = lineup_analysis['POSS'].round(1)
    lineup_analysis['OPP POSS'] = lineup_analysis['OPP POSS'].round(1)
    lineup_analysis['ORB%'] = lineup_analysis['ORB%'].round(1)
    lineup_analysis['DRB%'] = lineup_analysis['DRB%'].round(1)
    lineup_analysis['RB%'] = lineup_analysis['RB%'].round(1)
    lineup_analysis['TO%'] = lineup_analysis['TO%'].round(1)
    lineup_analysis['OPP ORB%'] = lineup_analysis['OPP ORB%'].round(1)
    lineup_analysis['OPP DRB%'] = lineup_analysis['OPP DRB%'].round(1)
    lineup_analysis['OPP RB%'] = lineup_analysis['OPP RB%'].round(1)
    lineup_analysis['OPP TO%'] = lineup_analysis['OPP TO%'].round(1)
    lineup_analysis['FTA%'] = lineup_analysis['FTA%'].round(1)
    lineup_analysis['OPP FTA%'] = lineup_analysis['OPP FTA%'].round(1)
    
    return lineup_analysis

def pbp_helper(df):
    lineup_data = []
    for x in pd.unique(df.Opponent):
        lineup_data.append( pbp_lineups( df[(df.Opponent == x)] ) )
        
    lineup_analysis = pd.concat(lineup_data)
    lineup_analysis = lineup_analysis.groupby('Lineup').sum()
        
    lineup_analysis = add_advanced(lineup_analysis)
    
    lineup_analysis = lineup_analysis.fillna(0).sort_values(by='MP', ascending=False)
    
    lineup_analysis['Lineup'] = lineup_analysis.index
    lineup_analysis.Lineup = lineup_analysis.Lineup.apply(lambda l : ", ".join([i.split(" ")[0] for i in l.split(", ")]))
    
    return lineup_analysis.set_index('Lineup')

def game_helper( game_pbp ):
    df = game_pbp.copy()
    df['Lineup'] = df.Opponent.apply(lambda i : ", ".join([i] * 5))

    lineup_data = []
    for x in pd.unique(df.Game):
        lineup_data.append( [pbp_lineups( df[(df.Game == x)] ), x] )

    lineup_analysis = pd.concat( [i[0] for i in lineup_data] )
    lineup_analysis['Game'] = [i[1] for i in lineup_data]
    lineup_analysis = lineup_analysis.groupby('Lineup').sum()

    lineup_analysis = add_advanced(lineup_analysis)
    
    lineup_analysis = lineup_analysis.sort_values(by='Game')
    lineup_analysis = lineup_analysis[['POSS', 'OPP POSS', 'PPP', 'OPP PPP', 'FTA%', 'OPP FTA%', 'PTS DIFF PER40', 'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 'TO%', 'OPP TO%']]
    
    return lineup_analysis.fillna(0)

def total_advanced(game_pbp):
    df = game_pbp.copy()
    df['Lineup'] = ", ".join(['Total']*5)
    
    lineup_analysis = pbp_lineups(df)
    
    lineup_analysis = add_advanced(lineup_analysis)
    
    lineup_analysis['POSS'] = round(40 * lineup_analysis['POSS'] / lineup_analysis['MP'], 1)
    lineup_analysis['OPP POSS'] = round(40 * lineup_analysis['OPP POSS'] / lineup_analysis['MP'], 1)
    
    lineup_analysis = lineup_analysis[['POSS', 'OPP POSS', 'PPP', 'OPP PPP', 'FTA%', 'OPP FTA%', 'PTS DIFF PER40', 'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 'TO%', 'OPP TO%']]
    lineup_analysis.index = ['Total']
    
    return lineup_analysis

In [46]:
team_start = "20:00"
current = uchicago
opponent = "Washington-St. Louis"

# lineups = pbp_helper( current )
game_advanced = pd.concat([game_helper( current ), total_advanced( current )])

In [41]:
new = current[(current.Opponent == opponent)]
new['length'] = new.Lineup.apply(lambda i : len(i.split(", ")))

new[(new.length != 5) & (new.Substitution == 0)]

,Date,Chicago,Opponent,Half,Time,Score,Team,Action,Substitution,Player,Lineup,Points,Game,length


In [42]:
pbp_helper( current[(current.Opponent == opponent)] ).MP.sum()

40.28

In [43]:
lineups

,MP,PTS,OPP PTS,+/-,AST,OREB,OPP OREB,DREB,OPP DREB,FGM,FGA,OPP FGM,OPP FGA,3PM,3PA,OPP 3PM,OPP 3PA,FT%,OPP FT%,FTA,OPP FTA,TO,OPP TO,STL,BLK,PF,OPP PF,FG%,OPP FG%,3P%,OPP 3P%,POSS,OPP POSS,PPP,OPP PPP,FTA%,OPP FTA%,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,OPP ORB%,OPP DRB%,OPP RB%,TO%,OPP TO%,POSS PER40
Lineup,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Alec, Arrish, Elliot, Ezra, Thomas",35.98,66,40,26,12,11,7,31,23,27,60,16,51,9,25,5,21,2.00,1.50,4,7,11,10,4,2,9,10,45.00,45.00,36.00,23.81,59.3,54.8,1.11,0.73,6.7,13.7,28.90,42,30,32.4,81.6,58.3,18.4,67.6,41.7,15.1,15.6,65.91
"Alec, Arrish, Ezra, Joe, Thomas",24.00,34,27,7,7,8,10,20,17,13,36,13,42,4,16,1,12,2.50,0.00,5,1,9,4,2,3,4,9,36.11,36.11,25.00,8.33,37.6,35.0,0.90,0.77,13.9,2.4,11.67,28,27,32.0,66.7,50.9,33.3,68.0,49.1,19.1,8.6,62.72
"Bryce, Elliot, Ezra, Jackson, Thomas",18.63,20,28,-8,4,2,0,14,14,6,23,11,24,3,13,3,6,2.50,1.33,6,6,10,7,3,2,11,6,26.09,26.09,23.08,50.00,32.3,32.3,0.62,0.87,26.1,25.0,-17.17,16,14,12.5,100.0,53.3,0.0,87.5,46.7,28.1,20.8,69.33
"Alec, Arrish, Eamonn, Elliot, Joe",12.43,17,28,-11,3,3,2,7,5,7,15,11,20,2,6,2,5,1.00,2.00,1,4,9,3,2,1,5,1,46.67,46.67,33.33,40.00,20.6,21.8,0.83,1.28,6.7,20.0,-35.39,10,7,37.5,77.8,58.8,22.2,62.5,41.2,36.8,12.1,66.22
"Bryce, Elliot, Jackson, Joe, Thomas",11.57,19,23,-4,1,1,4,9,8,6,14,9,20,0,2,4,8,3.00,0.50,7,3,2,2,1,1,3,6,42.86,42.86,0.00,50.00,17.4,18.6,1.09,1.24,50.0,15.0,-13.83,10,12,11.1,69.2,45.4,30.8,88.9,54.6,10.5,8.6,60.02
"Bryce, Eamonn, Ezra, Jackson, Joe",11.12,23,16,7,6,2,4,9,5,8,14,7,20,4,7,0,4,1.67,0.67,4,3,7,1,1,3,3,3,57.14,57.14,57.14,0.00,19.9,17.6,1.15,0.91,28.6,15.0,25.19,11,9,28.6,69.2,55.0,30.8,71.4,45.0,30.8,4.5,71.71
"Bryce, Elliot, Ezra, Josh, Thomas",10.10,16,14,2,5,3,1,6,7,7,17,7,14,1,5,0,2,1.00,0.00,1,0,3,5,4,1,1,6,41.18,41.18,20.00,0.00,16.7,17.3,0.96,0.81,5.9,0.0,7.92,9,8,30.0,85.7,52.9,14.3,70.0,47.1,14.7,26.3,66.31
"Eamonn, Joe, Josh, Leonardo, Skyler",9.78,22,14,8,6,3,2,6,3,10,16,4,12,2,3,2,5,0.00,2.00,0,4,2,4,2,1,3,0,62.50,62.50,66.67,40.00,14.4,15.1,1.53,0.93,0.0,33.3,32.71,9,5,50.0,75.0,64.3,25.0,50.0,35.7,11.1,22.5,58.88
"Alec, Arrish, Eamonn, Joe, Thomas",9.25,22,18,4,4,5,6,2,4,7,14,5,13,2,5,0,3,3.00,1.75,9,9,2,5,2,1,5,9,50.00,50.00,40.00,0.00,14.4,15.3,1.53,1.17,64.3,69.2,17.30,7,10,55.6,25.0,41.2,75.0,44.4,58.8,10.0,22.8,62.10


In [47]:
game_advanced

,POSS,OPP POSS,PPP,OPP PPP,FTA%,OPP FTA%,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,TO%,OPP TO%
Lake Forest,61.6,63.9,1.27,1.05,25.9,23.4,11.05,40,29,29.6,76.2,58.0,16.6,7.8
Albion,58.5,59.4,0.85,0.71,33.3,20.0,8.05,44,41,39.5,64.3,51.8,20.5,22.1
Mt. St. Joseph,57.8,55.5,1.11,1.06,7.8,21.6,5.03,39,30,40.5,75.0,56.5,12.0,15.2
Colorado College,66.5,65.0,1.01,1.06,23.5,18.8,-2.01,35,39,24.1,62.2,47.3,26.2,11.8
Colby,64.7,66.0,0.94,1.11,34.6,24.1,-12.22,38,30,29.0,90.0,55.9,19.1,16.7
Wheaton (Ill.),57.8,58.4,0.95,1.25,25.4,19.0,-18.17,38,32,34.2,78.1,54.3,16.4,7.4
Kalamazoo,59.4,60.5,1.21,1.04,18.6,32.0,9.06,47,25,50.0,80.6,65.3,20.0,18.6
Edgewood,67.8,68.7,1.25,1.05,70.2,27.3,13.13,32,34,26.3,78.6,48.5,7.4,20.6
Illinois Tech,59.0,63.7,1.05,0.93,34.0,16.4,3.02,40,31,23.3,80.5,56.3,16.1,12.1
Ill. Wesleyan,68.1,64.9,0.93,1.03,29.0,24.6,-3.58,38,45,23.3,70.0,45.8,13.6,15.1


In [45]:
games = []
for opp in pd.unique(current.Opponent):
    opp_game = pbp_helper( current[(current.Opponent == opp)] )
    opp_game['Opponent'] = opp
    games.append(opp_game)
    
games = pd.concat(games)

games['Lineup'] = games.index
games.Lineup = games.Lineup.apply(lambda l : ", ".join([i.split(" ")[0] for i in l.split(", ")]))

games.set_index('Lineup')

,MP,PTS,OPP PTS,+/-,AST,OREB,OPP OREB,DREB,OPP DREB,FGM,FGA,OPP FGM,OPP FGA,3PM,3PA,OPP 3PM,OPP 3PA,FT%,OPP FT%,FTA,OPP FTA,TO,OPP TO,STL,BLK,PF,OPP PF,FG%,OPP FG%,3P%,OPP 3P%,POSS,OPP POSS,PPP,OPP PPP,FTA%,OPP FTA%,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,OPP ORB%,OPP DRB%,OPP RB%,TO%,OPP TO%,POSS PER40,Opponent
Lineup,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Ellie, Grace, Mallory, Marissa, Sophia",8.23,14,10,4,2,2,5,4,3,5,10,4,13,1,1,0,1,0.75,1.00,4,2,4,4,3,2,2,2,50.00,50.00,100.00,0.00,13.2,12.4,1.06,0.81,40.0,15.4,19.43,6,8,40.0,44.4,42.9,55.6,60.0,57.1,25.4,22.4,64.18,North Central (Ill.)
"Bella, Ellie, Grace, Mallory, Marissa",6.95,12,14,-2,3,1,2,3,4,5,10,6,11,2,4,2,3,0.00,0.00,0,0,2,3,1,1,1,3,50.00,50.00,50.00,66.67,10.6,11.5,1.14,1.22,0.0,0.0,-11.51,4,6,20.0,60.0,40.0,40.0,80.0,60.0,16.7,21.4,60.78,North Central (Ill.)
"Ashley, Bella, Grace, Lindsey, Sophia",6.05,3,11,-8,0,2,3,5,5,1,8,4,13,0,2,3,5,0.50,0.00,2,0,2,2,1,1,3,2,12.50,12.50,0.00,60.00,8.5,11.5,0.35,0.95,25.0,0.0,-52.89,7,8,28.6,62.5,46.7,37.5,71.4,53.3,18.4,13.3,56.36,North Central (Ill.)
"Bella, Grace, Lindsey, Marissa, Sophia",4.03,8,5,3,1,1,1,5,1,4,6,2,8,0,1,1,3,0.00,0.00,0,0,4,1,0,1,1,1,66.67,66.67,0.00,33.33,8.6,7.7,0.93,0.65,0.0,0.0,29.75,6,2,50.0,83.3,75.0,16.7,50.0,25.0,40.0,11.1,85.69,North Central (Ill.)
"Ashley, Bella, Isabelle, Lindsey, Marissa",2.43,0,1,-1,0,2,1,2,1,0,4,0,4,0,3,0,2,0.00,0.50,0,2,1,1,0,0,2,0,0.00,0.00,0.00,0.00,2.9,4.7,0.00,0.21,0.0,50.0,-16.44,4,2,66.7,66.7,66.7,33.3,33.3,33.3,20.0,17.0,47.34,North Central (Ill.)
"Ashley, Bella, Lindsey, Marissa, Sophia",2.30,10,0,10,3,0,2,2,1,5,5,0,4,0,0,0,2,0.00,0.00,1,0,0,2,2,0,0,1,100.00,100.00,0.00,0.00,5.2,3.8,1.91,0.00,20.0,0.0,173.91,2,3,0.0,50.0,40.0,50.0,100.0,60.0,0.0,33.3,90.82,North Central (Ill.)
"Ashley, Bella, Grace, Lindsey, Marissa",1.50,4,0,4,0,2,2,2,1,2,5,0,4,0,2,0,1,0.00,0.00,0,0,0,0,0,1,2,0,40.00,40.00,0.00,0.00,2.9,1.9,1.39,0.00,0.0,0.0,106.67,4,3,66.7,50.0,57.1,50.0,33.3,42.9,0.0,0.0,76.80,North Central (Ill.)
"Ellie, Grace, Lindsey, Marissa, Sophia",1.20,3,0,3,1,0,0,1,1,1,2,0,1,1,1,0,1,0.00,0.00,0,0,1,1,0,0,0,0,50.00,50.00,100.00,0.00,2.9,1.9,1.04,0.00,0.0,0.0,100.00,1,1,0.0,100.0,50.0,0.0,100.0,50.0,33.3,50.0,96.00,North Central (Ill.)
"Ashley, Bella, Grace, Mallory, Marissa",0.80,3,0,3,0,0,0,1,0,1,1,0,1,0,0,0,1,1.00,0.00,1,0,1,0,0,0,1,1,100.00,100.00,0.00,0.00,2.3,1.0,1.28,0.00,100.0,0.0,150.00,1,0,0.0,100.0,100.0,0.0,0.0,0.0,41.0,0.0,117.12,North Central (Ill.)
